In [1]:
import numpy as np
import pandas as pd
from patsy import dmatrices
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn import metrics

/Library/Python/2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
# Load dataset
related =  pd.read_csv("./data_vectorised/related.csv")
infection =  pd.read_csv("./data_vectorised/infection.csv")
self =  pd.read_csv("./data_vectorised/self.csv")

## Data Exploration

In [3]:
#related.groupby('RESULT').mean()
#infection.groupby('RESULT').mean()
#self.groupby('RESULT').mean()

## Logistic Regression

In [4]:
related.head()

,index,flu,gett,swin,shot,s,nt,think,bird,sick,get,RESULT,ID
0,0,1,0,0,0,0,0,0,1,0,0,0,6004550306
1,1,1,1,0,0,0,0,0,0,0,0,0,6003455112
2,2,1,0,0,0,0,0,0,1,0,0,0,6002109706
3,3,2,0,1,0,0,0,0,0,0,0,1,5999019609
4,4,1,1,1,0,0,0,0,0,0,0,0,5990850113


In [5]:
y, X = dmatrices("RESULT ~ flu + gett + swin + shot + s + nt + think + bird + sick + get", related, return_type = 'dataframe')
# flatten y into a 1-D array
y = np.ravel(y)

In [6]:
# instantiate a logistic regression model, and fit with X and y
model = LogisticRegression()
model = model.fit(X,y)

# check the accuracy on the training set
model.score(X, y)

0.62325676784249384

## Model Evaluation Using a Validation Set

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
model2 = LogisticRegression()
model2.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

We now need to predict class labels for the test set. We will also generate the class probabilities, just to take a look.

In [9]:
# predict class labels for the test set
predicted = model2.predict(X_test)
print predicted

[ 1.  0.  1. ...,  0.  1.  0.]


In [10]:
# generate class probabilities
probs = model2.predict_proba(X_test)
print probs

[[ 0.47475099  0.52524901]
 [ 0.58802655  0.41197345]
 [ 0.41544288  0.58455712]
 ..., 
 [ 0.58802655  0.41197345]
 [ 0.40815592  0.59184408]
 [ 0.58802655  0.41197345]]


As can be seen, the classifier is predicting a 1 any time the probability in the second column is greater than 0.5.

Now let's generate some evaluation metrics.

In [12]:
# generate evaluation metrics
print metrics.accuracy_score(y_test, predicted)
print metrics.roc_auc_score(y_test, probs[:, 1])

0.620642515379
0.66584776784


The accuracy is 62%, which is the same as I experienced when training and predicting on the same data.
We can also see the confusion matrix and a classification report with other metrics

In [14]:
print metrics.confusion_matrix(y_test, predicted)
print metrics.classification_report(y_test, predicted)

[[419 241]
 [314 489]]
             precision    recall  f1-score   support

        0.0       0.57      0.63      0.60       660
        1.0       0.67      0.61      0.64       803

avg / total       0.63      0.62      0.62      1463



## Model Evaluation Using Cross-Validation

Now let's try 10-fold cross-validation, to see if the accuracy holds up more rigorously.

In [15]:
# evaluate the model using 10-fold cross-validation
scores = cross_val_score(LogisticRegression(), X, y, scoring='accuracy', cv=10)
print scores
print scores.mean()

NameError: name 'cross_val_score' is not defined

It's still performing at 62% accuracy'

## Predicting the Probability that a tweet is related to influenza

In [87]:
X = np.array([1,3,0,1,0,0,0,0,0,1,1])
X.reshape(-1, 1)
model.predict_proba(X)

/Library/Python/2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


array([[ 0.28514319,  0.71485681]])